In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath(".."))
from annnet.core.graph import Graph

### Composite indexing

In [4]:

G = Graph()

G.set_vertex_key("name", "sex")

m = G.get_or_create_vertex_by_attrs(name="a", sex="man")
w = G.get_or_create_vertex_by_attrs(name="a", sex="woman")

print(m, w, m == w)   # expect two different ids, False



kv:name='a'|sex='man' kv:name='a'|sex='woman' False


In [6]:
G.set_vertex_key("name")


ValueError: Composite key conflict for ('a',): kv:name='a'|sex='man' vs kv:name='a'|sex='woman'

In [ ]:
m = G.get_or_create_vertex_by_attrs(name="a", sex="man")
w = G.get_or_create_vertex_by_attrs(name="a", sex="woman")
print(m, w, m == w)   # same id, True

### Flexible directionality

In [9]:
G = Graph()
G.set_vertex_key("name","sex")

u = G.get_or_create_vertex_by_attrs(name="a", sex="man")
v = G.get_or_create_vertex_by_attrs(name="a", sex="woman")

# Helper to read signs
def signs(G, eid):
    s,t,_ = G.edge_definitions[eid]
    col = G.edge_to_idx[eid]
    si = G.entity_to_idx[s]; ti = G.entity_to_idx[t]
    M = G._matrix
    return M.get((si,col),0), M.get((ti,col),0)

# Edge-scope policy
e = G.add_edge(u, v,
    flexible={"var":"capacity", "threshold":0.7, "scope":"edge", "above":"s->t", "tie": "undirected"},
    capacity=0.5, weight=1.0)

print("init:", signs(G,e))            # expect (-1, +1)
G.set_edge_attrs(e, capacity=0.9)
print("after:", signs(G,e))           # expect (+1, -1)




init: (-1.0, 1.0)
after: (1.0, -1.0)


In [11]:
# Vertex-scope policy
G2 = Graph()
G2.set_vertex_key("name","sex")
s = G2.get_or_create_vertex_by_attrs(name="a", sex="man")
t = G2.get_or_create_vertex_by_attrs(name="a", sex="woman")
G2.set_vertex_attrs(s, temp=10.0)
G2.set_vertex_attrs(t, temp=20.0)

e2 = G2.add_edge(s, t,
    flexible={"var":"temp","threshold":0.0,"scope":"vertex","above":"s->t","tie":"undirected"},
    weight=1.0)

print("xs<xt:", signs(G2,e2))         # (-1, +1)
G2.set_vertex_attrs(s, temp=25.0)
print("xs>xt:", signs(G2,e2))         # (+1, -1)
G2.set_vertex_attrs(t, temp=25.0)
print("tie  :", signs(G2,e2))         # since "tie":"undirected", it becomes: (+1, +1)

xs<xt: (-1.0, 1.0)
xs>xt: (1.0, -1.0)
tie  : (1.0, 1.0)


### Kivela Multilayers

In [29]:
# ---------- build a tiny Kivela multilayer graph ----------

G = Graph(directed=True)

# 1) Define aspects and elementary layers
G.set_aspects(
    aspects=["time"],
    elem_layers={"time": ["t1", "t2"]},
)

# 2) Add vertices in the base graph
for u in ["A", "B", "C"]:
    G.add_vertex(u)

# 3) Declare node-layer presence (V_M)
for u in ["A", "B", "C"]:
    G.add_presence(u, ("t1",))
    G.add_presence(u, ("t2",))

# 4) Add Kivela edges (all go through incidence under the hood)

# intra edges in t1
G.add_intra_edge_nl("A", "B", ("t1",), weight=1.0)
G.add_intra_edge_nl("B", "C", ("t1",), weight=1.0)

# intra edges in t2
G.add_intra_edge_nl("A", "C", ("t2",), weight=2.0)

# inter-layer edge between A@t1 and B@t2
G.add_inter_edge_nl("A", ("t1",), "B", ("t2",), weight=0.5)

# coupling edges A: t1 <-> t2 and B: t1 <-> t2
G.add_coupling_edge_nl("A", ("t1",), ("t2",), weight=1.0)
G.add_coupling_edge_nl("B", ("t1",), ("t2",), weight=1.0)

# ---------- annotate layers and node-layer pairs ----------

# Elementary layer attributes, stored in G.layer_attributes
G.set_elementary_layer_attrs("time", "t1", order=1, name="early")
G.set_elementary_layer_attrs("time", "t2", order=2, name="late")

# Node-layer attributes, stored in _node_layer_attrs
G.set_node_layer_attrs("A", ("t1",), activity=0.2, color="blue")
G.set_node_layer_attrs("A", ("t2",), activity=0.9, color="red")
G.set_node_layer_attrs("B", ("t1",), activity=0.5)
G.set_node_layer_attrs("B", ("t2",), activity=0.7)

print("node-layer attrs A@t1:", G.get_node_layer_attrs("A", ("t1",)))
print("node-layer attrs A@t2:", G.get_node_layer_attrs("A", ("t2",)))
print()

# ---------- supra structures ----------

A_supra = G.supra_adjacency()
print("Supra adjacency shape:", A_supra.shape)
print("Supra adjacency matrix:\n", A_supra.toarray())
print()

L_comb = G.supra_laplacian(kind="comb")
L_norm = G.supra_laplacian(kind="norm")
print("Combinatorial Laplacian:\n", L_comb.toarray())
print()
print("Normalized Laplacian:\n", L_norm.toarray())
print()

print("Node-layer index (row -> (node, layer_tuple)):")
for i, (u, aa) in enumerate(G._row_to_nl):
    print(f"  {i}: {u} @ {aa}")
print()

deg = G.supra_degree()
print("Supra degrees per node-layer row:", deg)
print()

print("Participation coefficient per node:", G.participation_coefficient())
print("Versatility per node:", G.versatility())
print()

print("Layer attribute table (Polars):")
print(G.layer_attributes)


node-layer attrs A@t1: {'activity': 0.2, 'color': 'blue'}
node-layer attrs A@t2: {'activity': 0.9, 'color': 'red'}

Supra adjacency shape: (9, 9)
Supra adjacency matrix:
 [[0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  1.  0.  1.  0.5 0.  0.  0. ]
 [0.  1.  0.  0.  0.  0.  0.  0.  2. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  1.  0.  0.  0.  1.  0.  1.  0. ]
 [0.  0.5 0.  0.  1.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  1.  0.  0.  0.  0. ]
 [0.  0.  2.  0.  0.  0.  0.  0.  0. ]]

Combinatorial Laplacian:
 [[ 0.   0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   2.5 -1.   0.  -1.  -0.5  0.   0.   0. ]
 [ 0.  -1.   3.   0.   0.   0.   0.   0.  -2. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.  -1.   0.   0.   3.  -1.   0.  -1.   0. ]
 [ 0.  -0.5  0.   0.  -1.   1.5  0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.  -1.   0.   0.   1.   0. ]
 [ 0.   0.  -2.   0.   0.   0.   0.   0.   2. ]]

Normalized Lapla